In [1]:
import os
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import time, random

In [33]:
chromedriver = '/Users/lw/Desktop/chromedriver'
os.environ['webdriver.chrome.driver'] = \
        chromedriver
driver = webdriver.Chrome(chromedriver)
driver.implicitly_wait(20)

/var/folders/tv/8qzgqvq554vgrs4zb6bwbwlm0000gn/T/ipykernel_30890/3820478982.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


In [34]:
driver.get('https://www.chileconvencion.cl/documentos/')
driver.find_element_by_id('nav-iniciativas-tab').click() #Navigate to page containing documents of interest.

/var/folders/tv/8qzgqvq554vgrs4zb6bwbwlm0000gn/T/ipykernel_30890/1720728470.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('nav-iniciativas-tab').click() #Navigate to page containing documents of interest.


In [35]:
docs_table = []
for i in range(85):
    soup = bs(driver.page_source)
    [docs_table.append(doc) for doc in soup.find(id='tableIniciativas').find_all(class_='even')]
    [docs_table.append(doc) for doc in soup.find(id='tableIniciativas').find_all(class_='odd')]
    if i != 84:
        driver.find_element_by_id('tableIniciativas_next').click()
    

/var/folders/tv/8qzgqvq554vgrs4zb6bwbwlm0000gn/T/ipykernel_30890/2331300441.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('tableIniciativas_next').click()


In [36]:
urls = [doc.find_all('td')[5].find('a')['href'] for doc in docs_table]
nr = [doc.find_all('td')[0].text for doc in docs_table]

df = pd.DataFrame(urls, index=nr, columns=['doc_url'])
df['name'] = [doc.find_all('td')[1].text for doc in docs_table]
df['bulletin'] = [doc.find_all('td')[2].text for doc in docs_table]
df['topic'] = [doc.find_all('td')[3].text for doc in docs_table]
df['commission'] = [doc.find_all('td')[4].text for doc in docs_table]

In [37]:
print(df[~df.doc_url.str.contains('.pdf')].shape)
missing_url_index = df[~df.doc_url.str.contains('.pdf')].index
#print(docs_table[nr.index(missing_url_index)].find_all('td')[-1].find('a')) #Problem from the site
df.drop(index=missing_url_index, inplace=True, errors='ignore') #Drop initiative w missing url from df

(3, 5)


In [24]:
import re
#633-3 

In [64]:
df['name'] = df.name.apply(lambda x: re.sub(f'\xa0', ' ', x)) # invisible formatting that hinders search
df['name'] = df.name.apply(lambda x: re.sub(f'  ', ' ', x))

ind_df = df[df.name.str.lower().str.contains(r'ativa popular indígena')]
pop_df = df[df.name.str.lower().str.contains('iniciativa popular constituyente')]
multipl_df = df[df.name.str.lower().str.contains('iniciativas')] #Special PDFs that contain multiple proposals
conv_df = df[df.name.str.lower().str.contains('iva conv')|df.name.str.contains('iva Ind')]

In [65]:
info_dfs = [ind_df,pop_df,multipl_df,conv_df]
nr_docs = sum([spec_df.shape[0] for spec_df in info_dfs])
nr_docs, nr_docs==df.shape[0] # Sanity Check

(846, False)

In [66]:
[df.shape for df in info_dfs]

[(145, 5), (78, 5), (1, 5), (622, 5)]

In [67]:
conv_df.append(df.loc['18']).shape

/var/folders/tv/8qzgqvq554vgrs4zb6bwbwlm0000gn/T/ipykernel_30890/843948158.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  conv_df.append(df.loc['18']).shape


(623, 5)

In [74]:
unassigned = df[~df.index.isin(pd.concat(info_dfs).index)]

In [75]:
unassigned

,doc_url,name,bulletin,topic,commission
18,https://www.chileconvencion.cl/wp-content/uplo...,"ropone régimen tributario, fiscal ypresupuesta...",Boletín 633-3,"Régimen tributario, fiscal y presupuestario d...",Comisión sobre Forma de Estado


In [29]:
df[df.topic.str.contains('dica del agua')]

,doc_url,name,bulletin,topic,commission
221,https://www.chileconvencion.cl/wp-content/uplo...,Iniciativa Popular Constituyente sobre Natural...,Boletín 77-5 Popular,Naturaleza Jurídica del agua,Comisión sobre Medio Ambiente


In [30]:
df[df.topic.str.contains('Derecho humano al agua')]

,doc_url,name,bulletin,topic,commission
2,https://www.chileconvencion.cl/wp-content/uplo...,Iniciativa Convencional Constituyente que regu...,Boletín 649-5,Derecho humano al agua,Comisión sobre Medio Ambiente
690,https://www.chileconvencion.cl/wp-content/uplo...,Iniciativa Convencional Constituyente que decl...,Boiletín 164-4,Derecho humano al agua,Comisión de Derechos Fundamentales
715,https://www.chileconvencion.cl/wp-content/uplo...,Iniciativa Convencional Constituyente que cons...,Boletín 136-4,Derecho humano al agua,Comisión de Derechos Fundamentales


In [42]:
#SOBRE DEFENSORÍA DE LOS PUEBLOS
df[df.name.str.contains('Iniciativa Convencional')].shape

(615, 5)

In [32]:
df.head()

,doc_url,name,bulletin,topic,commission
2,https://www.chileconvencion.cl/wp-content/uplo...,Iniciativa Convencional Constituyente que regu...,Boletín 649-5,Derecho humano al agua,Comisión sobre Medio Ambiente
4,https://www.chileconvencion.cl/wp-content/uplo...,Iniciativa Convencional Constituyente que gara...,Boletín 647-5,"Océano, ecosistemas marinos y litoral costero",Comisión sobre Medio Ambiente
6,https://www.chileconvencion.cl/wp-content/uplo...,Iniciativa Convencional Constituyente que prop...,Boletín 645-5,Estatuto constitucional de la Energía,Comisión sobre Medio Ambiente
8,https://www.chileconvencion.cl/wp-content/uplo...,Iniciativa Convencional Constituyente que cons...,Boletín 643-6,Banco Central,Comisión sobre Sistemas de Justicia
10,https://www.chileconvencion.cl/wp-content/uplo...,Iniciativa Convencional Constituyente que regu...,Boletín 641-6,Contraloría General de la República,Comisión sobre Sistemas de Justicia
